In [1]:
# Для обработки данных
import pandas as pd
import numpy as np

# Для обработки аудио
import librosa
from pydub import AudioSegment

# Для скачивания аудио с ютуба и сохранения локально
import os
from pytube import Playlist

# Для разбиения на train/test
from sklearn.model_selection import train_test_split

## Extracting main audio features

In [37]:
def extract_audio_features(file_path):
    
    y, sr = librosa.load(file_path)

    rmse = librosa.feature.rms(y=y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

    features = {
        'rmse_mean': np.mean(rmse),
        'rmse_std': np.std(rmse),
        'spectral_centroid_mean': np.mean(spectral_centroid),
        'spectral_centroid_std': np.std(spectral_centroid),
        'spectral_bandwidth_mean': np.mean(spectral_bandwidth),
        'spectral_bandwidth_std': np.std(spectral_bandwidth),
        'rolloff_mean': np.mean(rolloff),
        'rolloff_std': np.std(rolloff),
        'zero_crossing_rate_mean': np.mean(zero_crossing_rate),
        'zero_crossing_rate_std': np.std(zero_crossing_rate),
    }

    for i in range(1, 41):
        features[f'mfcc{i}_mean'] = np.mean(mfcc[i-1])
        features[f'mfcc{i}_std'] = np.std(mfcc[i-1])

    for i in range(1, 13):
        features[f'chroma{i}_mean'] = np.mean(chroma[i-1])
        features[f'chroma{i}_std'] = np.std(chroma[i-1])

    for i in range(1, 7):
        features[f'tonnetz{i}_mean'] = np.mean(tonnetz[i-1])
        features[f'tonnetz{i}_std'] = np.std(tonnetz[i-1])

    features['chroma_cqt_mean'] = np.mean(chroma_cqt)
    features['chroma_cqt_std'] = np.std(chroma_cqt)

    features['spectral_contrast_mean'] = np.mean(spectral_contrast)
    features['spectral_contrast_std'] = np.std(spectral_contrast)

    return features

## Downloading youtube playlists with music

In [40]:
def download_audio_from_playlist(playlist_url, download_path):
    playlist = Playlist(playlist_url)
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    audio_files = []
    for video in playlist.videos:
        
        if len(audio_files) == 100:
            break
            
        print(f"Downloaded {len(audio_files)}")
    
        try:
            audio_stream = video.streams.filter(only_audio=True).first()
            output_path = audio_stream.download(output_path=download_path)
            base, ext = os.path.splitext(output_path)
            new_file = base + '.wav'
            AudioSegment.from_file(output_path).export(new_file, format="wav")
            os.remove(output_path)
        except Exception as e:
            print(f"Error: {e}")
            continue
        audio_files.append(new_file)
    return audio_files


rock_playlist = "https://youtube.com/playlist?list=PLyORnIW1xT6wFALM5dZlkFhOULbToFok3&si=Rd7b2VIsfiyE8uy9"
pop_playlist = "https://youtube.com/playlist?list=PLMC9KNkIncKtGvr2kFRuXBVmBev6cAJ2u&si=jO19iJUeivioOO16"
classical_playlist = "https://youtube.com/playlist?list=PL68AC80CBF3649BBB&si=Bq-KHuMN917vLZ_S"

download_dir = "../Data/downloaded_audio"

download_audio_from_playlist(rock_playlist, os.path.join(download_dir, "rock"))
download_audio_from_playlist(pop_playlist, os.path.join(download_dir, "pop"))
download_audio_from_playlist(classical_playlist, os.path.join(download_dir, "classical"))

Downloaded 0
Downloaded 1
Downloaded 2
Downloaded 3
Downloaded 4
Downloaded 5
Downloaded 6
Downloaded 7
Downloaded 8
Downloaded 9
Downloaded 10
Downloaded 11
Downloaded 12
Downloaded 13
Downloaded 14
Downloaded 15
Downloaded 16
Downloaded 17
Downloaded 18
Downloaded 19
Downloaded 20
Downloaded 21
Downloaded 22
Downloaded 23
Downloaded 24
Downloaded 25
Downloaded 26
Downloaded 27
Downloaded 28
Downloaded 29
Downloaded 30
Downloaded 31
Downloaded 32
Downloaded 33
Error: FRj0xuPF6Es is age restricted, and can't be accessed without logging in.
Downloaded 33
Downloaded 34
Downloaded 35
Downloaded 36
Downloaded 37
Downloaded 38
Downloaded 39
Error: bHi6NWXr0jI is age restricted, and can't be accessed without logging in.
Downloaded 39
Downloaded 40
Downloaded 41
Downloaded 42
Downloaded 43
Downloaded 44
Downloaded 45
Downloaded 46
Downloaded 47
Downloaded 48
Downloaded 49
Downloaded 50
Downloaded 51
Downloaded 52
Downloaded 53
Downloaded 54
Downloaded 55
Downloaded 56
Downloaded 57
Downloaded

['C:\\Users\\szabu\\PycharmProjects\\ML_basics_project\\Notebooks\\../Data/downloaded_audio\\classical\\Piotr Ilich Tchaikovsky - 1812 Overture (Finale).wav',
 'C:\\Users\\szabu\\PycharmProjects\\ML_basics_project\\Notebooks\\../Data/downloaded_audio\\classical\\Wolfgang Amadeus Mozart Eine kleine Nachtmusik.wav',
 'C:\\Users\\szabu\\PycharmProjects\\ML_basics_project\\Notebooks\\../Data/downloaded_audio\\classical\\Bach Toccata and Fugue in D minor BWV 565.wav',
 'C:\\Users\\szabu\\PycharmProjects\\ML_basics_project\\Notebooks\\../Data/downloaded_audio\\classical\\Rossini William Tell Overture Final.wav',
 'C:\\Users\\szabu\\PycharmProjects\\ML_basics_project\\Notebooks\\../Data/downloaded_audio\\classical\\Johann Pachelbel - Canon in D Major.wav',
 'C:\\Users\\szabu\\PycharmProjects\\ML_basics_project\\Notebooks\\../Data/downloaded_audio\\classical\\Johann Strauss II - The Blue Danube Waltz.wav',
 'C:\\Users\\szabu\\PycharmProjects\\ML_basics_project\\Notebooks\\../Data/downloaded_au

## Dividing audio into 3 seconds long snippets, after that doing train/test split (to avoid data leakage)

In [16]:
def get_files_and_labels(download_dir):
    genre_dirs = ["rock", "pop", "classical"]
    all_files = []
    all_labels = []
    for genre in genre_dirs:
        genre_path = os.path.join(download_dir, genre)
        if os.path.exists(genre_path):
            files = [os.path.join(genre_path, f) for f in os.listdir(genre_path) if f.endswith('.wav')]
            labels = [genre] * len(files)
            all_files.extend(files)
            all_labels.extend(labels)
    return all_files, all_labels

def split_audio(file_path, clip_length=3000):
    audio = AudioSegment.from_wav(file_path)
    clips = []
    for i in range(0, len(audio), clip_length):
        clip = audio[i:i + clip_length]
        if len(clip) == clip_length:
            clip_path = f"{file_path[:-4]}_clip_{i // clip_length}.wav"
            clip.export(clip_path, format="wav")
            clips.append(clip_path)
    return clips

def extract_audio_features_from_clips(clips, label):
    features = []
    for clip in clips:
        feature_dict = extract_audio_features(clip)
        feature_dict['label'] = label
        features.append(feature_dict)
        os.remove(clip)
    return features

def process_files(audio_files, labels):
    all_features = []
    for file_path, label in zip(audio_files, labels):
        print(f"Started processing {file_path}")
        clips = split_audio(file_path)
        features = extract_audio_features_from_clips(clips, label)
        all_features.extend(features)
    return all_features

download_dir = "../Data/downloaded_audio_cutting"
all_files, all_labels = get_files_and_labels(download_dir)

train_files, test_files, train_labels, test_labels = train_test_split(all_files, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

train_features = process_files(train_files, train_labels)
test_features = process_files(test_files, test_labels)

Started processing ../Data/downloaded_audio_cutting\pop\Shape of You.wav


C:\Users\szabu\anaconda3\envs\classical-ml\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Started processing ../Data/downloaded_audio_cutting\pop\golden hour (Cello Version).wav
Started processing ../Data/downloaded_audio_cutting\rock\Mr Brightside.wav
Started processing ../Data/downloaded_audio_cutting\classical\Imperial March or Darth Vaders Theme.wav
Started processing ../Data/downloaded_audio_cutting\pop\Sign of the Times (Official Video).wav
Started processing ../Data/downloaded_audio_cutting\rock\DEF MACHINE Feat Frank Nitt Diggidy VOIS Slam Dance ( OFFICIAL MUSIC VIDEO).wav
Started processing ../Data/downloaded_audio_cutting\rock\Highway to Hell (Official Video).wav
Started processing ../Data/downloaded_audio_cutting\rock\Patience.wav
Started processing ../Data/downloaded_audio_cutting\rock\Dont Let Me Down [blank].wav
Started processing ../Data/downloaded_audio_cutting\rock\Rescue Light.wav
Started processing ../Data/downloaded_audio_cutting\rock\Thunderstruck (Official Video).wav
Started processing ../Data/downloaded_audio_cutting\pop\STAY (Official Video).wav
Star

## Creating dataframes for later use

In [17]:
train_df = pd.DataFrame(train_features)
test_df = pd.DataFrame(test_features)

train_df.to_csv('train_features.csv', index=False)
test_df.to_csv('test_features.csv', index=False)

Feature extraction and dataset creation completed successfully.
